<img src="https://raw.githubusercontent.com/OpenEnergyPlatform/academy/develop/docs/data/img/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="75" width="75" align="left"/>
<img src="https://raw.githubusercontent.com/OpenEnergyPlatform/academy/develop/docs/data/img/rlilogo.png" alt="RLI" height="75" width="75" align="right"/>

# Uploading data to the OEP via script
<br><br>

If you are running this tutorial locally then use the requirements.txt to setup your environment. Otherwise make sure to install sqlalchemy>=1.3.0,<1.4 as version later then v1.4 do not support the oedialect.

In [ ]:
# pip install oedialect pandas "sqlalchemy~=1.3,<1.4" requests oep-client

import os
import getpass
import json

import pandas as pd
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
import oedialect
import requests
from oep_client import OepClient

## Reading data from a csv file into a dataframe

Pandas has a read_csv function which makes importing a csv-file rather comfortable. It reads csv into a DataFrame. By default, it assumes that the fields are comma-separated. Our example file has columns with semicolons as separators, so we have to specify this when reading the file. 

The example file for this tutorial ('DataTemplate.csv') is in a 'data' directory, one level above the file for this tutorial. Make sure to adapt the path to the file you're using if your file is located elsewhere.


In [ ]:
example_df = pd.read_csv('vg250-ew_2018-12-31.csv', encoding='utf8', sep=';', dtype={'RS': 'str'})
example_df.columns = map(str.lower, example_df.columns)

In [ ]:
print(example_df[:3])

# Using the API

If we want to upload data to the OEP we first need to connect to it, using our OEP user name and token. 
Note: You ca view your token on your OEP profile page after logging in.  

In [ ]:
# NEVER commit your token to a repository
# get your token from an environment variable
# or ask user
token = os.environ.get("OEP_API_TOKEN") or getpass.getpass('Token:')

In [ ]:
oep_url = 'https://openenergyplatform.org'
table = 'test_table_abschlussworkshop3'
schema = 'model_draft'

In [ ]:
# define table
# https://oep-data-interface.readthedocs.io/en/latest/api/how_to.html

data = { 
    "query": { 
        "columns": [
            { "name":"id", "data_type": "bigserial", "is_nullable": "NO" },
            { "name":"center_lon", "data_type": "numeric" },
            { "name":"center_lat", "data_type": "numeric" },
            { "name":"rs", "data_type": "varchar", "character_maximum_length": "50"},
            { "name":"gen", "data_type": "varchar", "character_maximum_length": "50" },
            { "name":"wsk", "data_type": "varchar", "character_maximum_length": "50" },
            { "name":"ewz", "data_type": "numeric" },
            { "name":"kfl", "data_type": "numeric" },
            { "name":"hat_bez", "data_type": "boolean" },
            { "name":"geometry_wkt", "data_type": "geometry" },
        ],
        "constraints": [ { "constraint_type": "PRIMARY KEY", "constraint_parameter": "id" } ] 
    }
}

## Creating the table

In [ ]:
requests.put(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/',
    json=data, verify=True,
    headers={'Authorization': 'Token {0}'.format(token)}
)

Delete table (do not delete if you want to proceed with next steps). PLese delte table after you finished with this tutorial.

In [ ]:
requests.delete(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table, verify=True,
    headers={'Authorization': f'Token {token}'}
)

## Uploading data to the table

In [ ]:
data = {
    "query": example_df.to_dict(orient='records')
}

In [ ]:
requests.post(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/rows/new',
    json=data, verify=True,
    headers={'Authorization': 'Token {0}'.format(token)}
)

# Add Metadata to table

In [ ]:
with open('metadata_testtable.json') as json_file:
    data = json.load(json_file)

In [ ]:
requests.post(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/meta/',
    json=data, verify=True,
    headers={'Authorization': 'Token {0}'.format(token)}
)

# Upload via sql-alchemy and oedialect

Now we'll create an sql-alchemy-engine. The engine is what 'speaks' oedialect to the data base api. We need to tell it where the data base is and pass our credentials.


In [ ]:
# Create Engine:
oed_string = f'postgresql+oedialect://:{token}@openenergyplatform.org'
table_name = 'test_table_abschlussworkshop3'
    
engine = sa.create_engine(oed_string)
metadata = sa.MetaData(bind=engine)

## Setup a Table

Define sqlalchemy columns.

In [ ]:
ExampleTable = sa.Table(
    table_name,
    metadata,
    sa.Column('id', sa.INTEGER),
    sa.Column('center_lon', sa.FLOAT),
    sa.Column('center_lat', sa.FLOAT),
    sa.Column('rs', sa.CHAR(5)),
    sa.Column('gen', sa.VARCHAR(50)),
    sa.Column('wsk', sa.DATE),
    sa.Column('ewz', sa.INTEGER),
    sa.Column('kfl', sa.FLOAT),
    sa.Column('hat_bez', sa.BOOLEAN),
    sa.Column('geometry_wkt', sa.VARCHAR(50)),
    schema=schema
)

## Create the new Table

Now we tell our engine to connect to the data base and create the defined table within the chosen schema. 

https://openenergyplatform.org/dataedit/view/model_draft/test_table_abschlussworkshop2

In [ ]:
conn = engine.connect()
print('Connection established')
if not engine.dialect.has_table(conn, table_name, schema):
    ExampleTable.create()
    print('Created table')
else:
    print('Table already exists')

## Delete the Table
Delete table (do not delete if you want to proceed with next steps). PLese delte table after you finished with this tutorial.

In [ ]:
# delete table
Session = sessionmaker(bind=engine)
session = Session()
conn = engine.connect()
print('Connection established')
if engine.dialect.has_table(conn, table_name, schema):
    ExampleTable.drop()
    print('Deleted table')
else:
    print('Something went wrong')

In [ ]:
session.close()

## Insert data into Table
 
Uploading the information from our DataFrame is now done with a single command. Uploading data in this way will always delete the content of the table and refill it with new values every time. If you change 'replace' to 'append', the data entries will be added to the preexisting ones. (Connecting and uploading may take a minute.)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

try:
    insert_statement = ExampleTable.insert().values(example_df.to_dict(orient='records'))
    session.execute(insert_statement)
    session.commit()
    print('Insert successful!')
except Exception as e:
    session.rollback()
    print('Insert incomplete!')
finally:
    session.close()

## Manually add single entries
You can also insert data manually into the table.

In [ ]:
manually_inserted_data = {
    'id': 99999,
    'center_lon': 12,
    'center_lat': 52,
    'rs': '99999',
    'gen': 'Atlantis',
    'wsk': '2021-01-20',
    'ewz': 1234567,
    'kfl': 0.0,
    'hat_bez': True,
    'geometry_wkt': 'POINT(12 52)'
}

In [ ]:
try:
    insert_statement = ExampleTable.insert().values([manually_inserted_data])
    session.execute(insert_statement)
    session.commit()
    print('Insert successful!')
except Exception as e:
    session.rollback()
    print('Insert incomplete!')
finally:
    session.close()

## Select from Table

Now  we can query our table to see if the data arrived.

In [ ]:
print(session.query(ExampleTable).all())
session.close()

## Storing Query Result in DataFrame
We can write the results of the query back into a DataFrame, where it's easier to handle.

In [ ]:
df = pd.DataFrame(session.query(ExampleTable).all())
print(df)